In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import os 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from os import system
system("cls")

0

In [2]:
os.chdir('/home/vidalyago_94/buckets/b1/' )

experimento='FE9252'

dataset = pd.read_csv("./exp/FE9251/dataset.csv.gz", low_memory=False)



In [3]:
#--------------------------------------------------------------
#Divido en entrenamiento y test
#train = dataset.loc[dataset["foto_mes"].isin([202102,202103])]
train = dataset.loc[dataset["foto_mes"] in [202102,202101,202007,202008,202103, 202104,202105]]
#test = dataset.loc[dataset["foto_mes"]==202105]
#test = dataset[dataset["foto_mes"]==202103]

# ELIMINO VALORES NULOS
train=train.fillna(train.mean()) # RELLENO CON LA MEDIA DE CADA COLUMNA LOS VALORES NULOS, LO NECESITA LA RED
#test = test.fillna(test.mean())

drop = ["foto_mes","numero_de_cliente","clase_ternaria"] #COLUMNAS A ELIMINAR

train = train[train.columns.drop( drop)] 
#test = test[test.columns.drop(drop)]

#ESCALO VARIABLES POR MÍNIMO Y MÁXIMO

scaler = MinMaxScaler()
scaler = scaler.fit(train) # OBTENGO VARIABLES DE ESCALADO EN TRAIN

train = scaler.transform(train) # LAS APLICO EN TRAIN
#test = scaler.transform(test)  # LAS APLICO EN TEST
variables = train.shape[1]  # OBTENGO VARIABLES TOTALES 

train = tf.cast(train, tf.float32)
#test = tf.cast(test, tf.float32)

C:\Users\vyago\AppData\Local\Temp\ipykernel_13796\4139612435.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train=train.fillna(train.mean()) # RELLENO CON LA MEDIA DE CADA COLUMNA LOS VALORES NULOS, LO NECESITA LA RED
C:\Users\vyago\AppData\Local\Temp\ipykernel_13796\4139612435.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test = test.fillna(test.mean())


In [4]:
class Autoencoder(Model):
  def __init__(self,variables):
    super(Autoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(variables,activation="relu"),
      layers.Dense(int(variables/2), activation="relu"),
      layers.Dense(int(variables/4), activation="relu"),
      layers.Dense(100, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(int(variables/4), activation="relu"),
      layers.Dense(int(variables/3), activation="relu"),
      layers.Dense(int(variables/2),activation="relu"),
      layers.Dense(variables, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

  def reductor(self,x):
      encoded = self.encoder(x)
      return encoded

In [5]:
autoencoder=Autoencoder(variables)

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

history = autoencoder.fit(x=train, y=train, 
            epochs=300, 
            batch_size=1048,
            shuffle=True)


310/310 [==============================] - 2s 3ms/step - loss: 0.0204 - val_loss: 0.0066


In [22]:
data=dataset[dataset.columns.drop(drop)]
data = data.fillna(data.mean())
data = scaler.transform(data)
features_nuevas = autoencoder.reductor(data)

In [29]:
dataframe = pd.DataFrame(features_nuevas)

In [ ]:
df = pd.concat([dataset,features_nuevas],axis=1)

if not os.path.isdir(f'./exp/{experimento}'):
    os.makedirs(f'./exp/{experimento}')

df.to_csv(f"./exp/{experimento}/dataset.csv.gz")